## Loading the data

The data for four sites is provided as a simple text file in CSV format.
Each file contains the dates of the peak bloom of the cherry trees at the respective site, alongside the geographical location of the site.

The six columns in each data file are

* _location_ a human-readable location identifier (`string`).
* _lat_ (approximate) latitude of the cherry trees (`double`).
* _long_ (approximate) longitude of the cherry trees (`double`).
* _alt_ (approximate) altitude of the cherry trees (`double`).
* _year_ year of the observation (`integer`).
* *bloom_date* date of peak bloom of the cherry trees (ISO 8601 date `string`). The "peak bloom date" may be defined differently for different sites
* *bloom_doy* days since January 1st of the year until peak bloom (`integer`). January 1st corresponds to `1`.


In [1]:
from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from meteostat import Daily, Stations

In [2]:
dataset_path = Path.cwd().parents[1] / "datasets/csv"

In [3]:
# Read and concatenate CSV files
cherry = pd.concat(
    [
        pd.read_csv(dataset_path/"washingtondc.csv"),
        pd.read_csv(dataset_path/"liestal.csv"),
        pd.read_csv(dataset_path/"kyoto.csv"),
        pd.read_csv(dataset_path/"vancouver.csv"),
    ],
    ignore_index=True,
)

In [ ]:
# Select the latest 3 observations for each location
latest_observations = cherry.groupby("location").tail(3)
latest_observations

## Visualizing the time series

In [ ]:
# Filter data for years since 1880
cherry_filtered = cherry[cherry["year"] >= 1880]

# Set up the plot grid with specific figure size
fig, axes = plt.subplots(
    nrows=1,
    ncols=len(cherry_filtered["location"].unique()),
    figsize=(15, 6),
    sharey=True,
)

# Plot for each location
for (location, data), ax in zip(cherry_filtered.groupby("location"), axes):
    ax.plot(
        data["year"], data["bloom_doy"], linestyle=":", color="gray"
    )
    ax.scatter(data["year"], data["bloom_doy"], color="blue", s=10)

    # Configure axis labels and title
    ax.set_title(location)
    ax.set_xlabel("Year")
    ax.set_xticks(range(1880, 2021, 20))
    ax.tick_params(axis="x", rotation=45)

# Common y-label for all subplots
axes[0].set_ylabel("Peak bloom (days since Jan 1st)")

# Adjust layout
plt.tight_layout()
plt.show()

## Predicting the peak bloom

A simple method to predict peak bloom date in the future is to fit a least-squares line through the observed dates and extrapolate the regression function.
We want to have a separate line for each location, hence we tell R to estimate _interaction_ effects.
We only use data from 1880 to fit the trends, as prior data may not be as reliable/relevant.

In [6]:
# Fit simple least-squares lines for all sites
ls_fit = smf.ols(
    "bloom_doy ~ location * year", data=cherry[cherry["year"] >= 1880]
).fit()

This simple linear regression functions suggest a trend toward earlier peak bloom at all 3 sites.
We can compute the actual predictions using the `predict()` function and 

In [ ]:
# Create a DataFrame to generate predictions for each location from 1880 to 2024
pred_years = pd.DataFrame(
    {
        "location": np.repeat(cherry["location"].unique(), 2024 - 1880 + 1),
        "year": list(range(1880, 2025)) * len(cherry["location"].unique()),
    }
)
pred_years

In [ ]:
# Compute predictions and 90% prediction intervals
pred_years["prediction"] = ls_fit.predict(pred_years)
pred_intervals = ls_fit.get_prediction(pred_years).summary_frame(alpha=0.1)
pred_years["lower"], pred_years["upper"] = (
    pred_intervals["obs_ci_lower"],
    pred_intervals["obs_ci_upper"],
)

# Filter cherry data and predictions for the specified years (2015 onward for most, 2021 for Vancouver)
cherry_filtered = cherry[
    (cherry["location"] == "vancouver") & (cherry["year"] >= 2021)
    | (cherry["location"] != "vancouver") & (cherry["year"] >= 2015)
]
pred_filtered = pred_years[
    (pred_years["location"] == "vancouver") & (pred_years["year"] >= 2021)
    | (pred_years["location"] != "vancouver") & (pred_years["year"] >= 2015)
]

# Plot the predictions with confidence intervals and actual observations
fig, axes = plt.subplots(
    nrows=1, ncols=len(cherry["location"].unique()), figsize=(8, 3), sharey=True
)

for (location, data), ax in zip(pred_filtered.groupby("location"), axes):
    # Plot the prediction line
    ax.plot(
        data["year"], data["prediction"], linewidth=1, color="blue", label="Prediction"
    )

    # Add the prediction interval as a ribbon
    ax.fill_between(
        data["year"],
        data["lower"],
        data["upper"],
        color="gray",
        alpha=0.3,
        label="90% Prediction Interval",
    )

    # Scatter plot of actual observations
    actual_data = cherry_filtered[cherry_filtered["location"] == location]
    ax.scatter(
        actual_data["year"], actual_data["bloom_doy"], color="black", label="Observed"
    )

    # Set axis labels, title, and ticks
    ax.set_title(location)
    ax.set_xlabel("Year")
    ax.set_xticks([2015, 2018, 2021, 2024])

# Set the y-label for the first subplot
axes[0].set_ylabel("Peak bloom (days since Jan 1st)")

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

Based on this very simple model, the peak bloom dates at the four sites are:

In [ ]:
# Helper function to convert day of the year to a date string
def doy_to_date(year, doy):
    date_obj = datetime.strptime(f"{year}-{doy}", "%Y-%j")
    return date_obj.strftime("%Y-%m-%d")


# Filter predictions for the year 2024 and round values
predictions_2024 = pred_years[pred_years["year"] == 2024].copy()
predictions_2024["prediction"] = predictions_2024["prediction"].round()
predictions_2024["lower"] = predictions_2024["lower"].apply(np.floor)
predictions_2024["upper"] = predictions_2024["upper"].apply(np.ceil)

# Convert prediction to date using the doy_to_date function
predictions_2024["prediction_date"] = predictions_2024.apply(
    lambda row: doy_to_date(row["year"], int(row["prediction"])), axis=1
)

# Display the predictions for 2024 with dates
predictions_2024[
    ["location", "year", "prediction", "lower", "upper", "prediction_date"]
]

## Extrapolating to Vancouver, BC

For the cherry trees in Vancouver, BC, few historical observations are available.
This shows in the simple analysis above in the very wide prediction interval.
The trees are located approximately at 49.2236916°N (latitude), -123.1636251°E (longitude), 24 meters above sea levels (altitude).
Casual observations have been recorded in the way of photos posted to the [VCBF Neighbourhood Blog for Kerrisdale](https://forums.botanicalgarden.ubc.ca/threads/kerrisdale.36008/).
You can search the forum for the keywords "Akebono" (i.e., the name of the cultivar) and "Maple Grove Park" (i.e., the location of the trees).

We need to *extrapolate* from what we have learned about the peak bloom dates in the other locations to Vancouver.
The simple model we have fitted above, however, does not allow us to transfer any knowledge from the other sites to Vancouver -- we have only used the history trend at the respective sites.

Although the climate in Vancouver is different from the other locations, the simplest way to borrow information from the other locations is to average across these three sites.
Hence, we want to fit a straight line through the peak bloom dates, ignoring the actual site:


In [ ]:
# Create weights: higher for Vancouver, lower for other locations
cherry_filtered = cherry[cherry['year'] >= 1880].copy()
cherry_filtered['weights'] = np.where(cherry_filtered['location'] == 'vancouver', 1.0, 0.2)

# Fit weighted least-squares linear regression model ignoring location
ls_fit_for_van = smf.wls("bloom_doy ~ year", data=cherry_filtered, weights=cherry_filtered['weights']).fit()

# Prepare DataFrame for predictions from 2022 to 2024 for Vancouver
predictions_vancouver = pd.DataFrame({'location': ['vancouver'] * 3, 'year': [2022, 2023, 2024]})

# Compute predictions and 90% prediction intervals
predictions_vancouver['prediction'] = ls_fit_for_van.predict(predictions_vancouver)
pred_intervals_van = ls_fit_for_van.get_prediction(predictions_vancouver).summary_frame(alpha=0.1)
predictions_vancouver['lower'], predictions_vancouver['upper'] = pred_intervals_van['obs_ci_lower'], pred_intervals_van['obs_ci_upper']

# Display the predictions
predictions_vancouver[['location', 'year', 'prediction', 'lower', 'upper']]


We can check the predictions against the data from previous competition years.

In [ ]:
# Join cherry data with Vancouver predictions for the years 2015 to 2023
cherry_with_predictions = cherry.merge(
    predictions_vancouver, on=["year", "location"], how="right"
)

# Filter data for the years 2015 up to 2023 for Vancouver
cherry_with_predictions = cherry_with_predictions[
    (
        (cherry_with_predictions["location"] == "vancouver")
        & (cherry_with_predictions["year"] >= 2015)
    )
]

# Plot
fig, ax = plt.subplots(figsize=(8, 3))

# Plot prediction line for Vancouver
ax.plot(
    cherry_with_predictions["year"],
    cherry_with_predictions["prediction"],
    linewidth=1,
    color="blue",
    label="Prediction",
)

# Add the prediction interval as a ribbon
ax.fill_between(
    cherry_with_predictions["year"],
    cherry_with_predictions["lower"],
    cherry_with_predictions["upper"],
    color="gray",
    alpha=0.3,
    label="90% Prediction Interval",
)

# Scatter plot for actual observed bloom dates
ax.scatter(
    cherry_with_predictions["year"],
    cherry_with_predictions["bloom_doy"],
    color="black",
    label="Observed",
)

# Set axis labels and ticks
ax.set_xlabel("Year")
ax.set_ylabel("Peak bloom (days since Jan 1st)")
ax.set_xticks(range(2022, 2025))
ax.set_title("Vancouver Cherry Blossom Peak Bloom Predictions (2022-2024)")

# Show legend and display the plot
ax.legend()
plt.tight_layout()
plt.show()

In [12]:
# Filter out old predictions for Vancouver and replace with the new ones
predictions = pd.concat([pred_years[pred_years['location'] != 'vancouver'], predictions_vancouver], ignore_index=True)

## Extrapolating to New York City, NY using USA-NPN data

Similar to Vancouver, BC, only few historical observations are available for our location in New York City, NY.
There are some historical observations dating back to 2019 in the data provided by USA-NPN.
The Washington Square Park has site id 32789 and the Yoshino cherry you should predict has species id 228.

In [ ]:
# Load and filter the USA-NPN data for New York City
nyc_data_npn = pd.read_csv(
    dataset_path / "USA-NPN_status_intensity_observations_data.csv"
)

# Filter for Washington Square Park (Site_ID 32789) and Yoshino cherry (Species_ID 228)
nyc_data_npn = nyc_data_npn[
    (nyc_data_npn["Site_ID"] == 32789) & (nyc_data_npn["Species_ID"] == 228)
]

# Convert the 'Observation_Date' to datetime format
nyc_data_npn["Observation_Date"] = pd.to_datetime(
    nyc_data_npn["Observation_Date"], format="%m/%d/%y"
)

# Display the first few rows of the filtered data to verify
nyc_data_npn.head()

This data, however, needs to be transformed as it only contains individual observations of the phenophase and not the actual peak bloom date.
For simplicity, we take the first day someone observed the flowers to be open as the peak bloom day.
This could be done in a more sophisticated way by also looking at the reported intensity value.


In [ ]:
# Ensure the data is sorted by Observation_Date
nyc_data = nyc_data_npn.sort_values('Observation_Date').copy()

# Add a 'year' column based on Observation_Date
nyc_data['year'] = nyc_data['Observation_Date'].dt.year

# Group by year and find the first day flowers were observed open (Phenophase_Status == 1)
nyc_data = (
    nyc_data.groupby('year')
    .apply(lambda df: df.loc[df['Phenophase_Status'] == 1].iloc[0] if not df[df['Phenophase_Status'] == 1].empty else None)
    .dropna()
)

# Extract bloom date information for each year
nyc_data['bloom_date'] = nyc_data['Observation_Date'].dt.strftime('%Y-%m-%d')
nyc_data['bloom_doy'] = nyc_data['Day_of_Year']

# Assign 'newyorkcity' as the location and reset index
nyc_data['location'] = 'newyorkcity'
nyc_data = nyc_data[['year', 'bloom_date', 'bloom_doy', 'location']].reset_index(drop=True)

# Combine this NYC data with the existing cherry data
cherry_with_nyc = pd.concat([cherry, nyc_data], ignore_index=True)

# Display the combined data
cherry_with_nyc.head()


Using the same steps as for Vancouver, BC, a simple linear model can be fitted.

In [ ]:
# Create weights: higher for NYC, lower for other sites
cherry_with_nyc['weights'] = np.where(cherry_with_nyc['location'] == 'newyorkcity', 1.0, 0.2)

# Fit the weighted least-squares linear regression model
ls_fit_for_nyc = smf.wls("bloom_doy ~ year", data=cherry_with_nyc[cherry_with_nyc['year'] >= 1880], 
                         weights=cherry_with_nyc[cherry_with_nyc['year'] >= 1880]['weights']).fit()

# Prepare a DataFrame for predictions for NYC from 2019 to 2024
predictions_nyc = pd.DataFrame({'location': ['newyorkcity'] * 6, 'year': list(range(2019, 2025))})

# Compute predictions and 90% prediction intervals
predictions_nyc['prediction'] = ls_fit_for_nyc.predict(predictions_nyc)
pred_intervals_nyc = ls_fit_for_nyc.get_prediction(predictions_nyc).summary_frame(alpha=0.1)
predictions_nyc['lower'], predictions_nyc['upper'] = pred_intervals_nyc['obs_ci_lower'], pred_intervals_nyc['obs_ci_upper']

# Display the predictions
predictions_nyc[['location', 'year', 'prediction', 'lower', 'upper']]


We can check the predictions against the data from previous competition years.

In [ ]:
# Merge actual bloom data with NYC predictions
cherry_with_predictions_nyc = cherry_with_nyc.merge(predictions_nyc, on=['year', 'location'], how='right')

# Filter data for plotting from 2015 to 2023 for NYC
cherry_with_predictions_nyc = cherry_with_predictions_nyc[
    ((cherry_with_predictions_nyc['location'] == 'newyorkcity') & (cherry_with_predictions_nyc['year'] >= 2015))
]

# Plot
fig, ax = plt.subplots(figsize=(8, 3))

# Plot the prediction line
ax.plot(cherry_with_predictions_nyc['year'], cherry_with_predictions_nyc['prediction'], linewidth=1, color='blue', label="Prediction")

# Add the 90% prediction interval as a ribbon
ax.fill_between(cherry_with_predictions_nyc['year'], cherry_with_predictions_nyc['lower'], cherry_with_predictions_nyc['upper'],
                color='gray', alpha=0.3, label="90% Prediction Interval")

# Plot actual observed bloom dates
ax.scatter(cherry_with_predictions_nyc['year'], cherry_with_predictions_nyc['bloom_doy'], color='black', label="Observed")

# Set axis labels, ticks, and title
ax.set_xlabel("Year")
ax.set_ylabel("Peak bloom (days since Jan 1st)")
ax.set_xticks(range(2019, 2025))
ax.set_title("Washington Square Park (NYC) Cherry Blossom Peak Bloom Predictions (2019-2024)")

# Show legend and display the plot
ax.legend()
plt.tight_layout()
plt.show()


If satisfied with the predictions, we can use them instead of the predictions from before.

In [ ]:
# Filter out existing NYC predictions and replace them with the new predictions
predictions = pd.concat([predictions[predictions['location'] != 'newyorkcity'], predictions_nyc], ignore_index=True)

# Display the updated predictions DataFrame
predictions.head()


## Preparing the submission file

Once we have the predictions for all four sites, we have to save them in the correct format for the competition.

In [ ]:
# Filter for the year 2024 and round prediction values for the submission
submission_predictions = predictions[predictions['year'] == 2024].copy()
submission_predictions['prediction'] = submission_predictions['prediction'].round()
submission_predictions['lower'] = submission_predictions['lower'].apply(np.floor).astype(int)
submission_predictions['upper'] = submission_predictions['upper'].apply(np.ceil).astype(int)

# Drop the 'year' column as specified
submission_predictions = submission_predictions.drop(columns='year')

# Display the submission predictions DataFrame to verify the format
submission_predictions.head()


In [ ]:
def get_station_by_coordinates(lat, lon):
    """Get the nearest weather station to given coordinates."""
    stations = Stations()
    stations = stations.nearby(lat, lon)
    station = stations.fetch(1)
    return station.index[0] if not station.empty else None

def get_temperature(station_id, start_date='1950-01-01', end_date='2023-01-31'):
    """
    Get seasonal temperature averages for a given station.
    
    Args:
        station_id: Meteostat station ID
        start_date: Start date for data collection
        end_date: End date for data collection
    
    Returns:
        DataFrame with yearly seasonal averages
    """
    # Convert dates to datetime
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    
    # Fetch daily data
    data = Daily(station_id, start, end)
    df = data.fetch()
    
    if df.empty:
        print(f"No data found for station {station_id}")
        return pd.DataFrame()
    
    # Create year and month columns
    df['year'] = df.index.year
    df['month'] = df.index.month
    
    # Adjust December to count towards next year's winter
    df['adjusted_year'] = df.apply(
        lambda x: x['year'] + 1 if x['month'] == 12 else x['year'], 
        axis=1
    )
    
    # Define seasons
    def get_season(month):
        if month in [12, 1, 2]:
            return 'Winter'
        elif month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Fall'
    
    df['season'] = df['month'].apply(get_season)
    
    # Calculate seasonal averages
    seasonal_avg = df.groupby(['adjusted_year', 'season'])['tmax'].mean().reset_index()
    seasonal_avg.columns = ['year', 'season', 'tmax_avg']
    
    # Convert temperature to tenths of degrees Celsius (to match R code)
    seasonal_avg['tmax_avg'] = seasonal_avg['tmax_avg'] * 10
    
    return seasonal_avg

def plot_temperature_trends(historic_temperatures):
    """Create a faceted plot of temperature trends."""
    # Set the style
    # plt.style.use('seaborn')
    
    # Create the faceted plot
    locations = historic_temperatures['location'].unique()
    seasons = ['Winter', 'Spring', 'Summer', 'Fall']
    
    fig, axes = plt.subplots(4, len(locations), figsize=(15, 12))
    fig.suptitle('Average Maximum Temperature by Season and Location', y=1.02)
    
    for i, season in enumerate(seasons):
        for j, location in enumerate(locations):
            data = historic_temperatures[
                (historic_temperatures['location'] == location) & 
                (historic_temperatures['season'] == season)
            ]
            
            axes[i, j].plot(data['year'], data['tmax_avg'])
            axes[i, j].set_xlim(1950, 2024)
            
            if i == 0:
                axes[i, j].set_title(location.title())
            if j == 0:
                axes[i, j].set_ylabel(f'{season}\nTemp (1/10 °C)')
            if i == 3:
                axes[i, j].set_xlabel('Year')
    
    plt.tight_layout()
    return fig

# Dictionary of cities and their coordinates
cities = {
    'washingtondc': (38.9072, -77.0369),
    'liestal': (47.4814, 7.7334),
    'kyoto': (35.0116, 135.7681),
    'vancouver': (49.2827, -123.1207)
}

# Get data for all cities
historic_temperatures = []

for city, coords in cities.items():
    station_id = get_station_by_coordinates(*coords)
    if station_id:
        temp_data = get_temperature(station_id)
        if not temp_data.empty:
            temp_data['location'] = city
            historic_temperatures.append(temp_data)

historic_temperatures = pd.concat(historic_temperatures, ignore_index=True)

# Create and display the plot
fig = plot_temperature_trends(historic_temperatures)
plt.show()

# Optional: Save the data
# historic_temperatures.to_csv('historic_temperatures.csv', index=False)